In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr                      
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import glob
from datetime import date, time, datetime 
import pandas as pd 
import netCDF4 as net
from netCDF4 import num2date
import scipy.io
from scipy.io import loadmat
from scipy import interpolate
import datetime as dt
import math

In [2]:
model = scipy.io.loadmat('Routine_v020721.mat')
alongtrack = xr.open_dataset('/Users/emmagurcan/Documents/JPL 2021/along-track data/Sentinel3A_NRT_2019_2020.nc')

In [3]:
# extract variables from the dataset
alongtrack_adt = alongtrack['adt']
alongtrack_lon = alongtrack['longitude']
alongtrack_lat = alongtrack['latitude']
alongtrack_time = alongtrack['time']

model_lon = model['lon']
model_lat = model['lat']
model_adt = model['adt2']
model_time = model['t']

alon = alongtrack_lon.values
alat = alongtrack_lat.values
atime = alongtrack_time.values

x = model_lon
y = model_lat
t = model_time
z = model_adt[:, :, :]

In [9]:
#slice alongtrack data to fit model data's coordinate restrictions
x_new = np.ndarray(shape=(342,72))
y_new= np.ndarray(shape=(342,72))
t_new = np.ndarray(shape=(14,342,72), dtype = 'datetime64[ns]')
t_new = np.where(t_new == t_new, pd.NaT, 0)
print('t_new initialized:')
print(t_new)

x_new[:] = np.NaN
y_new[:] = np.NaN

ix = 0
c = 0
for jy in range(len(alon[0,:])):
    while ix < len(alon):
        if alon[ix,jy] <= (np.max(model_lon)) and alon[ix,jy] >= (np.min(model_lon)) and alat[ix,jy] <= (np.max(model_lat)) and alat[ix,jy] >= (np.min(model_lat)):
            x_new[c,jy] = alon[ix,jy]
            y_new[c,jy] = alat[ix,jy]
            t_new[:,c,jy] = atime[:,ix,jy]
            
            # code to test
            #t_new[4,c,jy] = atime[4,ix,jy]
            #print('t_new:', t_new[4,c,jy])
            #print('alongtrack:', atime[4,ix,jy])
            c += 1
        ix += 1
    c = 0
    ix = 0 

t_new initialized:
[[[NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 [[NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 [[NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 ...

 [[NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 [[NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT Na

In [8]:
print('t_new:')
print(t_new[:,:,:])
print('alongtrack time')
print(atime[:,:,:])

t_new:
<class 'pandas._libs.tslibs.nattype.NaTType'>
[[[NaT None None ... NaT NaT 1567451651416199168]
  [NaT None None ... NaT NaT 1567451652416193792]
  [NaT None None ... NaT NaT 1567451653416198400]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 [[NaT None 1567577092020111872 ... NaT NaT 1569784454621603328]
  [NaT None 1567577093020106496 ... NaT NaT 1569784455621607936]
  [NaT 1567536484516440320 1567577094020111104 ... NaT NaT
   1569784456621602560]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 [[NaT None 1569909895000180224 ... NaT NaT 1572117254911945728]
  [NaT 1569869286631184384 1569909896000184832 ... NaT NaT
   1572117255911950336]
  [NaT 1569869287631188992 1569909897000179456 ... NaT NaT
   1572117256911944960]
  ...
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]
  [NaT NaT NaT ... NaT NaT NaT]]

 ...

 [[NaT None 1590905100621684736 ...

In [6]:
# THE REST OF THE INTERPOLATION CODE

# create third dimension for alongtrack lon and lat to fit with 3-D alongtrack time
#x_new2 = np.tile(x_new,(len(alongtrack['time']),1,1))
#y_new2 = np.tile(y_new,(len(alongtrack['time']),1,1))
# use t_new, make sure that t_new doesn't extend out past the model time period

#modify x_new and y_new such that they do not contain NaN
#     znew = np.empty(np.shape(z)); znew = np.NaN

#     # interpolate the model data to the alongtrack
#     d = (x[:,0],y[0,:], t)
#     ints = interpolate.RegularGridInterpolator(d, z)

#     # only input data where there are no NaNs
#     znew = ints((x_new[~np.isnan(x_new)], y_new[~np.isnan(y_new)]))

#     # Create figure
#     fig = plt.figure(figsize = (20, 5), dpi = 80)
#     ax = fig.add_subplot(projection=ccrs.PlateCarree())

#     ax.set_extent([np.min(x_new[~np.isnan(x_new)]), np.max(x_new[~np.isnan(x_new)]), np.min(y_new[~np.isnan(y_new)]), np.max(y_new[~np.isnan(y_new)])], crs = ccrs.PlateCarree())

#     gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                    linewidth=.5, color='gray', alpha=0.5, linestyle='--') # include gridlines on map
#     gl.top_labels = False     # don't want to plot labels on top side
#     gl.right_labels = False 
#     f = ax.scatter(x_new[~np.isnan(x_new)], y_new[~np.isnan(y_new)], c = znew, s = 5, cmap = 'coolwarm', vmin = 0.4, vmax = 0.9)
#     ax.add_feature(cfeature.LAND)
#     ax.add_feature(cfeature.COASTLINE) 

#     # Colorbar
#     fig.colorbar(f)

#     # Title
#     title = "Interpolated Absolute Dynamic Topography (in meters) from Model"
#     ax.set_title(title)

#     plt.show(